## Warning: this notebook is a mess

In [1]:
import os
import re
import csv
import fitz
import pandas as pd

In [100]:

def get_coordinates(pdf_path):
    doc = fitz.open(pdf_path)
    for page_num, page in enumerate(doc, start=1):
        text_instances = page.get_text("dict")["blocks"]
        for instance in text_instances:
            if "lines" in instance:  # Ensure it's a text block
                for line in instance["lines"]:
                    for span in line["spans"]:
                        print(f"Page {page_num}, Text: {span['text']}, Coordinates: {span['bbox']}")
                        # 'bbox' is the bounding box with coordinates (x0, y0, x1, y1)

    doc.close()

---

## Apr23_Directory_Recording_Studios.pdf

In [323]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            for span in line["spans"]:
                                # Ignore white text (color: 16777215)
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) < 5.5):
                                    if (span_count == 0) and (span["text"].isupper()):
                                        text += "##" + span["text"] + " "
                                    elif (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        text += span["text"].strip() + ": " 
                                    else:
                                        text += span["text"] + " " 
                                span_count += 1
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    #extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    for entry in extracted_list:
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                dict["Name"] = item.strip()
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list


In [324]:
"""
For Apr23_Directory_Recording_Studios.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "Apr23_Directory_Recording_Studios.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "Apr23_Directory_Recording_Studios.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

df["Email"] = df["Email"] + df["Gmail"]
df["Contact"] = df["Contact"] + df["Social"]
df["Web"] = df["Web"] + df["Website"] + df["Original Music Web"]
df["Basic Rate"] = df["Basic Rate"] + df["Basic Rates"] + df["Rates"] + df["Rate"]+ df["Studio Rates"] + df["Studio Rate"]
df["Format"] = df["Format"] + df["Formats"] + df["Main Format"] + df["Studio"] + df["Info"] + df["Gear"] + df["Studio Specs"] + df["Equipment"]
df["Services"] = df["Services"] + df["Special Services"]
df["Additional location"] = df["Additional location"] + df["Additional locations"] + df["Additional Locations"]
df["Notes"] = df["Notes"] + df["Comments"] + df["Description"]
df["Clients"] = df["Clients"] + df["Clients: Many song with all of these"]
keep = ["Name", "Address/Phone", "Email", "Web", "Basic Rate", "Format", "Contact", "Services", "Clients", "Additional location", "Notes"]
df = df.loc[:, keep]
df = df.iloc[1:, :]
df.to_csv(output_filepath)
df

,Name,Address/Phone,Email,Web,Basic Rate,Format,Contact,Services,Clients,Additional location,Notes
1,DAUPHIN STREET SOUND,"651 Dauphin St. Mobile, AL 36602 251-378-8028",info@dauphinstreetsound.com,facebook.com/dauphinstreetsound,Call or email for rates,,,,,,
2,FAME STUDIOS,"603 E. Avalon Ave. Muscle Shoals, AL 35661 2...",info@famestudios.com,fame2.com,please call for info,Pro Tools and vintage analog,,,,,
3,RIVERLAND RECORDING STUDIOS,"513 Talucah Rd. Valhermoso Springs, AL 35775...",,facebook.com/riverlandrecording,$65/hr to $80/hr. Contact us,"Alesis HD 24, Pro Tools LE",,,,,
4,"SOUND CELL/SMITH MUSIC GRP, INC.","601 Meridian St. Huntsville, AL 35801 256-53...",soundcell@soundcell.com,soundcell.com,please call for info,Pro Tools,,,,,
5,SOUND OF BIRMINGHAM RECORDING,"3625 5th Ave. S. Birmingham, AL 35222 205-59...",don@soundofbirmingham.com,soundofbirmingham.com,please call for info,,Don Mosley,,,,
...,...,...,...,...,...,...,...,...,...,...,...
588,ORBIT AUDIO,"Seattle, WA 206-381-1244",orbitaudio@gmail.com,orbitaudiorocks.com,"$95/Hr, $750/day",Digital/Analog,,,,,
589,JAMIE PECK PRODUCTIONS,"216 Longvue Acres Rd. Wheeling, WV 26003 304...",,jamiepeckproductions.com,please call for info,digital Pro Tools HD,Jamie Peck,,,,
590,RHL AUDIO,703-628-3015,chris@rhlaudio.com,rhlaudio.com,please call for info,,Chris Murphy,,,,
591,BLAST HOUSE STUDIOS,"1117 Jonathon Dr. Madison, WI 53713 608-276...",mike@madisonmusicfoundry.com,blasthousestudios.com,call for rates,see website for equipment list,,,,,


---

## Feb24_ProducerDirectory.pdf

In [321]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range, appending "##" to certain lines.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False  # Track if the previous line had "##" appended
    last_line_was_newline= True  # Track if the previous line was an empty line

    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:
                    if "lines" in text_dict:
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) == 6.75):
                                    if span.get("flags", 0) != 20 or ':' in span["text"]:
                                        append_hash = False
                                    if (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        line_text += span["text"].strip() + ": " 
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip() + "\n"
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line

                            # Reset the flag if the last line was not a newline
                            if text.endswith("\n"):
                                last_line_was_newline = True
                            else:
                                last_line_was_newline = False
                        text += "\n"  # Add an extra line break for readability between blocks

    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    #extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    affiliation = ""
    section = ""
    for entry in extracted_list:
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                if "825 RECORDS" in item:
                    section = "INDEPENDENT PRODUCERS"
                    affiliation = "INDEPENDENT PRODUCERS"
                elif "WOODY" in item:
                    section = "INDEPENDENT ENGINEERS"
                    affiliation = "INDEPENDENT ENGINEERS"

                if item.isupper():
                    affiliation = item.strip()
                    dict["Name"] = item.strip()
                    dict["Affiliation"] = section
                else:
                    dict["Name"] = item.strip()
                    dict["Affiliation"] = affiliation
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list


In [322]:
"""
For Feb24_ProducerDirectory.pdf

# General structure:
    - Producer name in bold
    - Followed by job titles in parentheses (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "Feb24_ProducerDirectory.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "Feb24_ProducerDirectory.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

df["Web"] = df["Web"] + df["Website"] + df["Music"]
df["Format"] = df["Format"] + df["Main Format"] + df["Facility, Analog Tape Restoration"]
df["Additional location"] = df["Additional location"] + df["Additional locations"]
df["Clients"] = df["Notable Projects"] + df["Notable projects"] + df["Notable Projects/Clients"] + df["Notable Projects and Clients"] + df["Production"] + df["Recent Projects"] + df["Clients"] + df["Notable Associates"] + df["Credits"] + df["Client Roster includes"]
df["Services"] = df["Services"] + df["Specialities"]
df["Styles"] = df["Styles"] + df["Genre"]
df["Phone"] = df["Phone"] + df["Call"]
df["Contact"] = df["Contact"] + df["Studio Manager"]
df["Notes"] = df["Description"]
df["Producers/Mixers/Engineers"] = df["Mixing/Engineering"] + df["Producers"] + df["Producers, Mixers, Engineers"] + df["Notable Engineers"]
keep = ["Name", "Affiliation", "Address/Phone", "Phone", "Email", "Web", "Format", "Contact", "Services", "Clients", "Styles", "Producers/Mixers/Engineers", "Additional location", "Notes"]
df = df.loc[:, keep]
df = df.iloc[1:, :]
df.to_csv(output_filepath)

df

#extract_text(input_filepath)

,Name,Affiliation,Address/Phone,Phone,Email,Web,Format,Contact,Services,Clients,Styles,Producers/Mixers/Engineers,Additional location,Notes
1,AAM,,"Advanced Alternative Media 270 Lafayette St., ...",,info@aaminc.com,aaminc.com,,,,,,,"5979 W. 3rd St., Ste. 204 Los Angeles, CA 900...",
2,"JOE DAMBROSIO MANAGEMENT, INC.",,"875 Mamaroneck Ave., Ste. 403 Mamaroneck, NY 1...",,info@jdmanagement.com,jdmanagement.comPeople: Isaiah Abolin Jay Dufo...,,,,,,,,
3,GPS/GLOBAL POSITIONING SERVICES,,"3435 Ocean Park Blvd. Santa Monica, CA 90405 3...",,info@globalpositioningservices.net,globalpositioningservices.net,,"Jerimaya Grabher, JW Johnson,Colin Ramsay",,,,,,
4,Michael Brauer,GPS/GLOBAL POSITIONING SERVICES,"(mixer, engineer)",,,,,,,"Coldplay, John Mayer,Luther Vandross, Aretha F...",,,,
5,Billy Bush,GPS/GLOBAL POSITIONING SERVICES,"(producer, engineer, mixer)",,,,,,,"The Naked & Famous,Garbage, Angus and Julia St...",,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,DUSTY WAKEMAN,INDEPENDENT ENGINEERS,Mojave Audio 1118 Chestnut St. Unit 104 Burba...,,dusty@mojaveaudio.com,mojaveaudio.com,,,,"Dwight Yoakam, LucindaWilliams, Jim Lauderdale...","americana, rock, country, world,jazz",,,
352,TOM WEIR,INDEPENDENT ENGINEERS,"4412 Whitsett Ave. Studio City, CA 91604 818-5...",,bookings@studiocitysound.com,studiocitysound.com,,,,"Rod Stewart, Scott Wei-land, Heather Youmans, ...",all,,,
353,MICHAEL WOODRUM,INDEPENDENT ENGINEERS,818-848-3393,,michael@woodrumproductions.com,woodrumproductions.com,,,,"Prince, Eric Clapton,Joss Stone, Snoop Dogg, W...",all,,,
354,TOBY WRIGHT,INDEPENDENT ENGINEERS,,,,tobywrightmusic.com,,,,"Alice In Chains, 3 DoorsDown, The Wallfowers, ...","Rock, Metal, Pop, Reggae, Jazz",,,


---

## Nov23_Publishing_Guide

In [319]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) == 7):
                                    if (span.get("flags", 0) != 20) or (':' in span["text"]) or (not span["text"].isupper()):
                                        append_hash = False
                                    if (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        line_text += span["text"].strip() + ": " 
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip() + "\n"
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    #extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    for entry in extracted_list:
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                dict["Name"] = item.strip()
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list


In [320]:
"""
For Nov23_Publishing_Guide.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "Nov23_Publishing_Guide.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "Nov23_Publishing_Guide.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

df["Additional location"] = df["Additional location"] + df["Additional locations"]
df["Styles"] = df["Styles"] + df["All Styles"]
df["Published"] = df["Published"] + df["Notes"] + df["Credits"]
df["How to Submit"] = df["How to Submit"] + df["How To Submit"]
keep = ["Name", "Address/Phone", "Email", "Fax", "Web", "How to Submit", "Contact", "Styles", "Published", "Additional location", "Notes"]
df = df.loc[:, keep]
df = df.iloc[1:, :]
df.to_csv(output_filepath)
df

#print(extract_text(input_filepath))

,Name,Address/Phone,Email,Fax,Web,How to Submit,Contact,Styles,Published,Additional location,Notes
1,3 RING CIRCUS MUSIC,"2209 Grantland Ave Nashville, TN 37204",admin@3ringcircusmusic.com,,3ringcircusmusic.com,,,,,,
2,ABET MUSIC,"411 E. Huntington Dr., Ste. 107 Arcadia, CA ...",info@abetpublishing.com,,alfred.com,no unsolicited material,,"easy listening, chill, rock, world music, alte...",,,
3,"ANOTHER VICTORY, INC.","CONCORD MUSIC 5750 Wilshire Blvd., Suite 450...",,,"concord.com/labels/victory-records, facebook.c...",no unsolicited material,,,,"10 Lea Ave., #300 Nashville, TN 37210 629-4...",
4,ANTHEM ENTERTAINMENT,"120 Bremner Blvd, #2900 Toronto, ON, Canada ...",,,anthementertainment.com/music- publishing,,,,,"Nashville 462 Humphreys Street Nashville, T...",
5,A WRITERS PARADISE,"Nashville, TN 615-852-8297",info@bigdealmusic.com,,bigdealmusic.com,no unsolicited material,Stacy Hogan,,,,
...,...,...,...,...,...,...,...,...,...,...,...
99,"HARRY FOX AGENCY, INC., THE (HFA)","40 Wall St., 6th Fl. New York, NY 10005 646...",,,harryfox.com,,,,,,
100,NATIONAL MUSIC PUBLISHERS,"ASSOCIATION (NMPA) 1900 N St. NW, Suite 500 ...",,,nmpa.org,,,,,,
101,SESAC,"35 Music Sq. E. Nashville, TN 37203 615-320...",,,sesac.com,,,all styles,,"Santa Monica 2150 Colorado Ave., Suite 150 ...",
102,"SOUNDEXCHANGE, INC.","733 10th St., N.W., 10th Fl. Washington, D.C...",info@soundexchange.com,,Soundexchange.com,,,,,,


---

## July23_Managers_Guide

In [179]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) >= 6.69):
                                    if (span.get("flags", 0) != 20) or (':' in span["text"]) or (not span["text"].isupper()):
                                        append_hash = False
                                    if (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        line_text += span["text"].strip() + ": " 
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip() + "\n"
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    #extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    role = "Manager"
    for entry in extracted_list:
        entry = entry.strip('\n\n')
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                if "25 LIVE" in item:
                    print("DEBUG 25 LIVE")
                    role = "Booking Agent"

                dict["Name"] = item.strip()
                dict["Role"] = role
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list

In [180]:
"""
For July23_Managers_Guide.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "July23_Managers_Guide.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "July23_Managers_Guide.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

df["Additional location"] = df["Additional location"] + df["Additional locations"] + df['Additional Location']
df["Styles"] = df["Styles"] + df["Style"]
df["Clients"] = df["Clients"] + df["Client"] + df["Published"] + df["Past Acts include"]
df["Email"] = df["Email"] + df["Info"]
df["Services"] = df["Services"] + df["Specialties"] + df["Styles/Specialties"]
keep = ['Name', 'Role', 'Address/Phone', 'Email', 'Fax', 'Web', 'Styles',
       'Additional location', 'Contact', 'Clients', 'Services',
       'How to Submit', 'A&R/Manager', 'Booking Agent', 'Notes']
df = df.loc[:, keep]
df.to_csv(output_filepath)
df

#extract_text(input_filepath)

DEBUG 25 LIVE


,Name,Role,Address/Phone,Email,Web,Styles,Additional location,Contact,Clients,Additional locations,...,Published,How to Submit,Info,Styles/Specialties,Specialties,Past Acts include,A&R/Manager,Booking Agent,Notes,Fax
0,5B ARTIST MANAGEMENT,Manager,"220 36th St., Ste. B442 Brooklyn, NY 11232 ...",info@5bam.com,5bam.com,"Metal, Rock, Alt. *No unsolicited material","12021 W. Jefferson Blvd. Culver City, CA 900...",,,,...,,,,,,,,,,
1,AAM,Manager,"270 Lafayette St., Ste 605 New York, NY 1001...",info@aaminc.com,aaminc.com,"Alt., Rock, Pop, Indie",,Matthew Clayman,"Call for roster *No unsolicited material, repr...","5979 W. 3rd St., Ste. 204 Los Angeles, CA 90...",...,,,,,,,,,,
2,ARSLANIAN & ASSOCIATES,Manager,"5419 Hollywood Blvd Suite C717 Hollywood, C...",oscar@discoverhollywood.com,arslanianassociates.com/who.html,Classic Rock,,Oscar Arslanian,"Fabian, Chris Montez, Little Peggy March, Kath...",,...,,,,,,,,,,
3,"ELLEN AZORIN, MANAGER, PEDRO",Manager,"GIRAUDO TANGO QUARTET 157 W. 79th St., Ste. ...",ellenazorin@gmail.com,pedrogiraudo.com,Performance Argentine Tango,,,,,...,,,,,,,,,,
4,THE BABBLE BOUTIQUE,Manager,,Azalee@Azalee.Maslow.com,azaleemaslow.com/the-babble- boutique,All,,Azalee Maslow,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,SPECTRUM TALENT AGENCY,Booking Agent,212-268-0404,marc@spectrumtalentagency.com,spectrumtalentagency.com,"pop, R&B, dance, hip-hop, theatrical",,Marc Katz,info upon request *No unsolicited material,,...,,,,,,,,,,
175,THIRD COAST TALENT,Booking Agent,"P.O. Box170 Chapmansboro, TN 37035 615-685-...",carrie@thirdcoasttalent.com,thirdcoasttalent.com,Country,,,see website for complete roster *No unsolicite...,,...,,,,,,,,,,
176,UNITED TALENT AGENCY,Booking Agent,"9336 Civic Center Dr. Beverly Hills, CA 9021...",,unitedtalent.com,,,,See site for a list *No unsolicited material,"New York, NY 888 7th Ave., Seventh FL. New ...",...,,,,,,,,,,
177,UNCLE BOOKING,Booking Agent,"5438 Winding Way Dr. Houston, TX 77091 510-...",erik@unclebooking.com,unclebooking.com,,,,"Kevin Barnes, Bottomless Pit, CEX, Citay, the ...",,...,,,,,,,,,,


---

## Jun23_Vocal_Coach_Guide

In [317]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) == 5.75):
                                    if (span.get("flags", 0) != 20 and span['text'] != ' ') or (':' in span["text"]) or (not span["text"].isupper() and span['text'] != ' '):
                                        append_hash = False
                                    if (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        line_text += span["text"].strip() + ": " 
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip() + "\n"
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    for entry in extracted_list:
        entry = entry.strip('\n\n')
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                dict["Name"] = item.strip()
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list

In [318]:
"""
For Jun23_Vocal_Coach_Guide.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "Jun23_Vocal_Coach_Guide.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "Jun23_Vocal_Coach_Guide.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

df["Basic rate"] = df["Basic rate"] + df["Basic Rate"] + df["Rates"]
df["Additional location"] = df["Additional location"] + df["Additional locations"] + df["Locations"]
df["Styles"] = df["Styles"] + df["Style"]
df["Services"] = df["Services"] + df["Duration"]
keep = ['Name', 'Address/Phone', 'Email', 'Web', 'Basic Rate', 'Clients', 'Contact', 'Services', 'Additional location', 'Styles', 'Credits', 'Notes']
df = df.loc[:, keep]
df = df.iloc[1:, :]
df.to_csv(output_filepath)
df

#extract_text(input_filepath)

,Name,Address/Phone,Email,Web,Basic Rate,Clients,Contact,Services,Additional location,Styles,Credits,Notes
1,ANGEL DIVA MUSIC,"Jan Linder Koda Los Angeles, CA 818-888-588...",jan@angeldivamusic.com,angeldivamusic.com,please call for info,"Holly Robinson-Pete , Tim Fagan, David Hassel...",,,,,,Author of the book Once More With Feeling. Gra...
2,APPELL VOICE STUDIO,"#1 Pegasus Dr. Trabuco Canyon, CA 92679 949...",vocalcoach.tea@gmail.com,appellvoicestudio.com,,,Thomas Appell,"vocal coaching, record production, music produ...",,,,Author of Can You Sing a HIGH C Without Strai...
3,ARTIST VOCAL DEVELOPMENT,"West Hills, CA 818-430-3254",sbensusen@socal.rr.com,steviebensusen.com (click on Artist Vocal Deve...,Please call for rates,"Jordan Knight (New Kids On The Block), Nichole...",,Stevie Bensusen is a Multiplatinum Vocal Produ...,,,,
4,AUDREY BABCOCK,"Los Angeles, CA 917-572-4494 Singer-Songwri...",audrey@audreybabcock.com,audreybabcock.com Notes : As a teacher and coa...,,,,,,,,
5,BEVERLY BREMERS VOICERCISE,"5159 Shotwell Street Woodstock, GA 30188 94...",bev@voicercise.net,voicercise.net,$75/hr. for private instruction online only vi...,all levels - adults and children,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
72,UAF SUMMER SESSIONS &,LIFE-LONG LEARNING University of Alaska - Fa...,summer@alaska.edu,uaf.edu/summer,please call for info,,,,,,,
73,VALERIE FAHREN,"The L.A. Valley Area, CA 818-815-8584",ValerieFahren@aol.com,valeriefahren.com,,,Valerie Fahren,,,,,
74,VOCAL STYLINGS,"Culver City, CA 310-737-9387",info@vocalstylings.com,vocalstylings.com,please call for info,"Private lessons, group sessions, in studio voc...",Lisa Cushing,,,,,
75,VOICE LESSONS HOUSTON,"13505 Westheimer Rd. Houston, TX 77077 713-...",winnie@voicelessonshouston.com,voicelessonshouston.com,please call for info,,,,,,,


---

## Mar24_Indie_Guide

In [315]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) >= 5.59 and span.get("size", 0) <= 5.61):
                                    if (span.get("flags", 0) != 20) or (':' in span["text"]) or (not span["text"].isupper()):
                                        append_hash = False
                                    if (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        line_text += span["text"].strip() + ": " 
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip() + "\n"
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    #extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    type = "Indie Label"
    for entry in extracted_list:
        entry = entry.strip('\n\n')
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                if "SEO EXPERTS" in item:
                    print("1 SEO EXPERTS")
                    type = "Marketer/Promoter"
                elif "1888 MEDIA" in item:
                    print("1888 MEDIA")
                    type = "Publicist"
                elif "ADOBE GRAPHICS" in item:
                    print("ADOBE GRAPHICS")
                    type = "Merch & Swag"
                elif "BANDCAMP" in item:
                    print("BANDCAMP")
                    type = "Promo Site"
                elif "CD BABY" in item:
                    print("CD BABY")
                    type = "Distribution/Online Retail"
                elif "ARTIST GROWTH" in item:
                    print("ARTIST GROWTH")
                    type = "Fan Mail/Career Management"
                elif "CARDSCASHREWARDS" in item:
                    print("CARDSCASHREWARDS")
                    type = "Networking/Social/Gig"
                elif "BROADJAM" in item:
                    print("BROADJAM")
                    type = "Licensing"
                elif "DATAMUSICATA" in item:
                    print("DATAMUSICATA")
                    type = "Information/Opportunities"
                elif "2DOPEBOYZ" in item:
                    print("2DOPEBOYZ")
                    type = "Blog"
                elif "ARTIST SHARE" in item:
                    print("ARTIST SHARE")
                    type = "Other"

                dict["Name"] = item.strip()
                dict["Type"] = type
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list

In [316]:
"""
For Mar24_Indie_Guide.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "Mar24_Indie_Guide.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "Mar24_Indie_Guide.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

"""df["Additional location"] = df["Additional location"] + df["Additional locations"] + df['Additional Location']
df["Styles"] = df["Styles"] + df["Style"]
df["Clients"] = df["Clients"] + df["Client"] + df["Published"] + df["Past Acts include"]
df["Email"] = df["Email"] + df["Info"]
df["Services"] = df["Services"] + df["Specialties"] + df["Styles/Specialties"]
"""

df["Email"] = df["Email"] + df["E-mail"]
df["Web"] = df["Web"] + df["Website"]
df["Clients"] = df["Clients"] + df["Client"] + df["Current Clients"] + df["Notable Artists"] + df["Roster"] + df["Roster/Notable Projects"] + df["Published"] + df["#1 Albums in 2020"]
df["Additional locations"] = df["Additional locations"] + df["Additional Locations"] + df["Additional Location"] + df["Additional location"]
df["Styles/Specialties"] = df["Styles/Specialties"] + df["Styles"] + df["Styles/Specialities"] + df["Specialities"] + df["Syles/Specialties"] + df["Formats/Specialties"] + df["Specialties"] + df["Specialty"]
df["Notes"] = df["Notes"] + df["Note"]
df["Phone"] = df["Phone"] + df["Hollywood"] + df["Nashville"]
df["Services"] = df["Services"] + df["Services Provided"] + df["Service"]
keep = ['Name', 'Type', 'Address/Phone', 'Email', 'Phone', 'Fax', 'Web', 'Contact', 'Styles/Specialties', 'Services', 'Genres', 'Clients', 'Additional locations', 'How to Submit', "Notes"]
df = df.loc[:, keep]
df = df.iloc[1:, :]
df.to_csv(output_filepath)
df

#print(extract_text(input_filepath))

1 SEO EXPERTS
CARDSCASHREWARDS
1888 MEDIA
ADOBE GRAPHICS
BANDCAMP
CD BABY
CARDSCASHREWARDS
ARTIST GROWTH
CARDSCASHREWARDS
BROADJAM
CARDSCASHREWARDS
DATAMUSICATA
2DOPEBOYZ
DATAMUSICATA
ARTIST SHARE


,Name,Type,Address/Phone,Email,Phone,Fax,Web,Contact,Styles/Specialties,Services,Genres,Clients,Additional locations,How to Submit,Notes
1,1-2-3-4 GO! RECORDS,Indie Label,"420 40th Street #5 Oakland, CA 94609 510-98...",store@1234gorecords.com,,,1234gorecords.com *not actively accepting demos,,,,,,,,
2,300 ENTERTAINMENT,Indie Label,"450 West 14 New York, NY 10014",info@threehundred.biz,,,300ent.com,,,,,"Megan Thee Stallion, Young Thug, Fetty Wap, Ma...",,,
3,4AD RECORDS,Indie Label,"17-19 Alma Rd. London, SW 18, 1AA, U.K. +44...",demos@4ad.com,,,4ad.com,,,,,"Adrianne Lenker, Aldous Harding, Anjimile, Atl...","134 Grand Street New York, NY 10013 212-9...",,
4,88RISING,Indie Label,"New York, NY & Los Angeles, CA",shop@88rising.com,,,88rising.com,Sean Miyashiro,,,,"Joji, Warren Hue, The Jackson Wang, Bibi, Niki",,,
5,"825 RECORDS, INC.",Indie Label,"8225 5 Brooklyn, NY 11209 347-240-5417",mattyamendola@825records.com,,,825Records.com,,"Artist development, solo artists, singer/songw...",,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,ARTIST SHARE,Other,,,,,artistshare.com,,,,,,,,fan funding like Kickstarter
565,KICKSTARTER,Other,,,,,kickstarter.com,,,,,,,,fund raising website for DIY artists
566,SONGDEX,Other,,,,,songdex.com,,,,,,,,Industry song database
567,SOUND EXCHANGE,Other,,,,,soundexchange.com,,,,,,,,collecting online royalties


---

## Jan24_GuideA&R

In [313]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) >= 4.92 and span.get("size", 0) <= 4.93):
                                    if (span.get("flags", 0) != 20 and span['text'] != ' ') or (':' in span["text"]) or (not span["text"].isupper() and span['text'] != ' '):
                                        append_hash = False
                                    if (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        line_text += span["text"].strip() + ": " 
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip() + "\n"
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    for entry in extracted_list:
        entry = entry.strip('\n\n')
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                dict["Name"] = item.strip()
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list

In [314]:
"""
For Jan24_GuideA&Re.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "Jan24_GuideA&R.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "Jan24_GuideA&R.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

df["Email"] = df["Email"] + df["Email to"]
df["Web"] = df["Web"] + df["Website"]
df["Styles/Specialties"] = df["Styles/Specialties"] + df["Style/Specialties"] + df["Styles/Specialities"] + df["Styles"] + df["Genre Styles"]
df["Clients"] = df["Clients"] + df["Roster"] + df["Client List"]
df["Additional locations"] = df["Additional locations"] + df["Additional location"]
df["How to Submit"] = df["How to Submit"] + df["Submissions"]
keep = ['Name', 'Address/Phone', 'Phone', 'Email', 'Web', 'Clients', 'Contact', 'Services', 'Styles/Specialties', 'Distribution', 'How to Submit','Additional locations']
df = df.loc[:, keep]
df = df.iloc[1:, :]
df.to_csv(output_filepath)
df

#extract_text(input_filepath)

,Name,Address/Phone,Phone,Email,Web,Clients,Contact,Services,Styles/Specialties,Distribution,How to Submit,Additional locations
1,1-2-3-4 GO!,"420 40th St., Ste. #5 Oakland, CA 94609 510...",,store@1234gorecords.com,1234gorecords.com,,,,"rock, punk *Do not accept unsolicited material",,,
2,4AD RECORDS,"134 Grand St. New York, NY 10013 212-995-58...",,"4ad@4ad.com, edhorrox@4ad.com",4ad.com,"the National, Deerhunter, Efterklang, Aldous H...","Ed Horrox, A&R",,rock/indie,,,"17-19 Alma Rd., SW18 1AA, UK 44-208-870-9724"
3,ACTIVATE ENTERTAINMENT,5062 Lankershim Blvd.-174 N. Hollywood CA 91...,,submissions@2Activate.com,,,James Arthur,,,,"Urban, Soul/R&B, HipHop, Pop, Hard Rock, Count...",
4,ALLIGATOR RECORDS,"Attn: New Material P.O. Box 60234 Chicago, ...",,info@allig.com,alligator.com,,,,"blues, roots, contemporary blues *Please inclu...",,,
5,AMATHUS MUSIC,"P.O. Box 611 Rockville Centre, NY 11571",,"info@amathusmusic.com,",amathusmusic.com,,,,"dance, electronic, pop *Do not accept unsolici...",,,
...,...,...,...,...,...,...,...,...,...,...,...,...
144,WARNER MUSIC GROUP,"1633 Broadway New York, NY 10019 212-275-2000",,,,,,,,,,
145,Web:: wmg.com,"*No unsolicited material Mike Caren, Preside...",,,,,,,,,,
146,WARNER MUSIC LATINA,"555 Washington Ave., 4th Fl. Miami Beach, FL...",,gabriella.martinez@wmg.com,facebook.com/warnermusiclat *No unsolicited ma...,,,,,,,
147,WICKED COOL RECORDS,"434 6th Ave., Ste. 6R New York, NY 10011 34...",,info@wickedcoolrecords.com scott@wickedcoolrec...,wickedcoolrecords.com,,,,garage rock,,,


---

## May23_Digital_Resources_Guide

In [308]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:

                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) <= 10):
                                    """# Flagging the section headers
                                    if (span["text"].isupper()) and (span.get("color", 0) == 2236191) and (span.get("size", 0) == 20):
                                        if span_count == 0:
                                            line_text += "##" + span["text"] + " " 
                                        elif span_count == len(line["spans"]):
                                            line_text += span["text"] + "##"
                                        else:
                                            line_text += span["text"] + " " """

                                    # Flagging the company/service name    
                                    if (span.get("flags", 0) != 20) and (not span["text"].isupper()) and (span.get("color", 0) != 12472091) and (span.get("size", 0) != 10):
                                        append_hash = False
                                        line_text += span["text"] + " " 
                                    
                                    # Flagging the attribute
                                    elif (':' in span["text"]) and (span.get("flags", 0) == 20) and (span.get("color", 0) == 12472091) and (span.get("size", 0) == 8.25):
                                        line_text += span["text"].strip() + ": " 

                                    # For all other text
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip()
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    #extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    type = "Social Media & Digital Marketing Tools"
    for entry in extracted_list:
        entry = entry.strip('\n\n')
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                if "BANDZOOGLE" == item:
                    type = "Web Design"
                elif "ArrangeMe" in item:
                    type = "Legal Services/Advice"
                elif "ARTISTSHARE" in item:
                    type = "Fundraising/Financial Matters"
                elif item == "FLUENCE":
                    type = "Music Review/Opinion/Feedback"
                elif "ADAPTR" in item:
                    type = "Song Submissions/Placements"
                elif "AIRBIT" in item:
                    type = "Beat Selling Websites"
                elif "GREAT AMERICAN SONG CONTEST" in item:
                    type = "Song Competitions"
                elif "AIRBNB" in item:
                    type = "On the Road: Lodging"
                elif "AIRTABLE" in item:
                    type = "Artist Collaborations"
                elif "CHARTMETRIC" in item:
                    type = "Music Sales Tracking"
                elif "ARI'S TAKE ACADEMY" in item:
                    type = "Online Instruction"
                elif "DISCOGS" in item:
                    type = "Music Credits Database"
                elif "BANDSINTOWN" in item:
                    type = "Tour Dates Calendar"
                elif "DREAMSTAGE" in item:
                    type = "Live Streaming"
                elif "9 CAFEPRESS" in item:
                    type = "T-Shirts & More: Merch & Manufacturing"
                elif item == "AIMI":
                    type = "Music Distribution & Film Licensing"
                elif "DROPP TV" in item:
                    type = "Monetized Video Posting"

                dict["Name"] = item.strip()
                dict["Type"] = type
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list

In [309]:
"""
For May23_Digital_Resources_Guide.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "May23_Digital_Resources_Guide.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "May23_Digital_Resources_Guide.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)
df = df.iloc[1:, :]
df.to_csv(output_filepath)

df

#extract_text(input_filepath)

,Name,Type,Web,Services,Fee,Email,Phone,Features,Fees,Summary,Prizes,Categories,Contact
1,BITLY,Social Media & Digital Marketing Tools,bitly.com,"URL Shortener, Custom Branded URLs, API & Link...",,,,,,,,,
2,CANVA,Social Media & Digital Marketing Tools,canva.com,Graphic-design tool website. Design presentati...,,,,,,,,,
3,CINDERBLOCK,Social Media & Digital Marketing Tools,Cinderblock.TV,Video streaming app that is built for music vi...,,,,,,,,,
4,CROWDMOUTH,Social Media & Digital Marketing Tools,crowdmouth.com,Crowdmouth is a direct-to-fan platform empower...,,,,,,,,,
5,FORTUNES,Social Media & Digital Marketing Tools,fortunes.io,"Cross-platform music data analytics, career re...",,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,THE ORCHARD,Music Distribution & Film Licensing,theorchard.com,"distribution, marketing, data analytics, perfo...",,communications@theorchard.com,,,,,,,
190,TUNECORE,Music Distribution & Film Licensing,tunecore.com,"Vinyl pressing and distribution, physical copi...",,,,,,,,,
191,UNITEDMASTERS,Music Distribution & Film Licensing,unitedmasters.com,"Distribution, marketing, analytics","$100 monthly payment made through paypal, Unit...",info@unitedmasters.com,,,,,,,
192,DROPP TV,Monetized Video Posting,dropp.tv,Dropp Tv unleashes the power of the creator th...,,support@dropp.tv,,,,,,,


---

## Oct23_Mastering_Guide

In [310]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) == 6.25):
                                    if (span.get("flags", 0) != 20 and span['text'] != ' ') or (':' in span["text"]) or (not span["text"].isupper() and span['text'] != ' '):
                                        append_hash = False
                                    if (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        line_text += span["text"].strip() + ": " 
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip() + "\n"
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    #extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    for entry in extracted_list:
        entry = entry.strip('\n\n')
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                dict["Name"] = item.strip()
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list

In [311]:
"""
For Oct23_Mastering_Guide.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "Oct23_Mastering_Guide.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "Oct23_Mastering_Guide.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

"""df["Email"] = df["Email"] + df["Email to"]
df["Web"] = df["Web"] + df["Website"]
df["Styles/Specialties"] = df["Styles/Specialties"] + df["Style/Specialties"] + df["Styles/Specialities"] + df["Styles"] + df["Genre Styles"]
df["Clients"] = df["Clients"] + df["Roster"] + df["Client List"]
df["Additional locations"] = df["Additional locations"] + df["Additional location"]
df["How to Submit"] = df["How to Submit"] + df["Submissions"]
"""
df["Basic Rates"] = df["Basic Rates"] + df["Basic rates"] + df["Basic Rate"] + df["Rates"] + df["Rate"]
df["Clients"] = df["Clients"] + df["Previous Clients"] + df["Notable Projects"]
df["Format"] = df["Format"] + df["Gear"]
keep = ['Name', 'Address/Phone', 'Email', 'Web', 'Contact', 'Clients', 'Credits', 'Services', 'Format', 'Basic Rates', 'Specialization', 'Additional location', 'Masterclass', 'Note']
df = df.loc[:, keep]
df = df.iloc[1:, :]
df.to_csv(output_filepath)
df

#extract_text(input_filepath)

,Name,Address/Phone,Email,Web,Contact,Clients,Credits,Services,Format,Basic Rates,Specialization,Additional location,Masterclass,Note
1,BATES BROTHERS RECORDING,"3427 Davey Allison Blvd., Ste. 101 Hueytown,...",bbrbates@yahoo.com,batesbrothersrecording.com,Eric Bates,,,,,please see web for info,,,,
2,SOUND OF BIRMINGHAM,"3625 5th Ave. S. Birmingham, AL 35222 205-5...",don@soundofbirmingham.com,soundofbirmingham.com,,,,,,,,,,"35 years + exp., please call for info"
3,1ST CHOICE MASTERING,"650 N. 157th Drive Goodyear, AZ 85338 623-...",info@billkeis.com,billkeis.com/producer,,,,see web for info,,,,,,
4,ALLUSION STUDIOS,"248 W. Elm St. Tucson, AZ 85705 520-622-3895",contact@allusionstudios.com,allusionstudios.com,,,,,,please call for info,,,,
5,M L E STUDIOS,"PO Box 27017 Prescott Valley, AZ 86312 866-...",mail@majorlabelmusic.com,"majorlabelmusic.com, facebook.com/mlestudios",Col. Darryl Harrelson,,,,,$45.00/hr or flat rate per song/ album small i...,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,BLUE ALLEY STUDIOS,"347 Dunkard Ave Westover, WV. 26501 304-598...",jswiger@bluesalleymusic.com,bluesalleymusic.com,,,,,,,,,,
222,SUNNY DAY SOUND,"Charleston, WV 304-881-4110",sunnydaysound@gmail.com,sunnydaysound.com,,,,,,,,,,
223,ZONE 8 RECORDING,"1973 Main Street Granville, WV. 26534 304-5...",zone8recording@yahoo.com,zone8recording.com,Mark Poole,,,,,please call for info,,,,
224,FRESH COAST,"818 N. 109th St. Milwaukee, WI 53226 414-77...",,freshcoast.com,,,,,,please call for info,,,,


---

## Jun23_MuEd_Guide

In [327]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) >= 5.7 and span.get("size", 0) <= 5.72):
                                    if (span.get("flags", 0) != 20 and span['text'] != ' ') or (':' in span["text"]) or (not span["text"].isupper() and span['text'] != ' '):
                                        append_hash = False
                                    if (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        line_text += span["text"].strip() + ": " 
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip() + "\n"
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    #extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    for entry in extracted_list:
        entry = entry.strip('\n\n')
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                dict["Name"] = item.strip()
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list

In [342]:
"""
For Jun23_MuEd_Guide.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "Jun23_MuEd_Guide.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "Jun23_MuEd_Guide.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

df["Notes"] = df["Notes"] + df["Note"]
df["Basic Rate"] = df["Basic Rate"] + df["Basic rate"] + df["Rates"] + df["Cost"] + df["lessons and ensemble lessons. Cost"]
df["Degrees/Certificates Offered"] = df["Degrees/Certificates Offered"] + df["Degree"] + df["Degrees"] + df["Degree offered"] + df["Degree/Certification"] + df["Degrees/Certifications"]
df["Services"] = df["Services"] + df["Programs"] + df["Program"] + df["Instruments"] + df["Performance"] + df["Courses"]
df["Contact"] = df["Contact"] + df["Director"]
df["Additional locations"] = df["Additional locations"] + df["Additional Location"] + df["Additional location"] + df["Satellite Facility"] + df["Satellite locations"] + df["Studio locations"]
keep = ['Name', 'Address/Phone', 'Phone', 'Fax', 'Mailing', 'Email', 'Web', 'Contact', 'Additional locations', 'Degrees/Certificates Offered', 'Program and Facilities Description', 'Services', 'Duration', 'Basic Rate', 'Clients', 'Notes']
df = df.loc[:, keep]
df = df.iloc[1:, :]
df.to_csv(output_filepath)
df

#extract_text(input_filepath)

,Name,Address/Phone,Phone,Fax,Mailing,Email,Web,Contact,Additional locations,Degrees/Certificates Offered,Program and Facilities Description,Services,Duration,Basic Rate,Clients,Notes
1,"ARS NOVA, INC.","2828 Drake Ave., S.W. Huntsville, AL 35802",,,"P.O. Box 14248 Huntsville, AL 35815 256-883-11...",arsnovaschool81@bellsouth.net,arsnovahsv.com,,,,,,,please call or see web for info,,
2,UNIVERSITY OF ALABAMA,"810 Second Ave. Tuscaloosa, AL 35487 205-34...",,,,ssnead@ua.edu,music.ua.edu,"Charles Skip Snead, School of Music Director",,,,,,please call or see web for info,,
3,UNIVERSITY OF NORTH ALABAMA,Department of Entertainment Industry 1 Harri...,,,,ragarfrerick@una.edu,una.edu/entertainment,,,,,Bachelor of Arts or Bachelor of Science in Ent...,,,,
4,UNIVERSITY OF ALASKA,"1708 Tanana Loop, Ste. 201 Fairbanks, AK 997...",,,,,uaf.edu/music,,,,,,,please call or see web for info,,
5,"BILL KEIS MUSIC, INC.","650 N. 157th Drive Goodyear, AZ 85338 818-2...",,,,info@billkeis.com Web : billkeis.com,,,,,,,,,,Subjects covered include: Repertoire (learning...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,UNIVERSITY OF TORONTO,Faculty of Music Admission Office - Room 145 ...,,,,,music.utoronto.ca,,,Two-year Masters in Music Technology and Digit...,,,,,,
418,BERKLEE ONLINE,1-617-747-2146,,,,advisors@online.berklee.edu,online.berklee.edu,,,,,,,,,
419,EDDIE HADDAD,Lead Guitar Instructor Guitar Mastery Method...,,,,eddie@guitarmasterymethod.com,guitarmasterymethod.com,,,,,,,,,
420,FULL SAIL UNIVERSITY,844-706-0683,,,,,fullsail.edu/campus-and-online/online- degrees...,,,,,,,,,


---

## Jan24_GuideAttorneys

In [347]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) >= 4.9 and span.get("size", 0) <= 5):
                                    if (span.get("flags", 0) != 20 and span['text'] != ' ') or (':' in span["text"]) or (not span["text"].isupper() and span['text'] != ' '):
                                        append_hash = False
                                    if (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        line_text += span["text"].strip() + ": " 
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip() + "\n"
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    #extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    for entry in extracted_list:
        entry = entry.strip('\n\n')
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                dict["Name"] = item.strip()
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list

In [354]:
"""
For Jan24_GuideAttorneys.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "Jan24_GuideAttorneys.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "Jan24_GuideAttorneys.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

"""df["Notes"] = df["Notes"] + df["Note"]
df["Basic Rate"] = df["Basic Rate"] + df["Basic rate"] + df["Rates"] + df["Cost"] + df["lessons and ensemble lessons. Cost"]
df["Degrees/Certificates Offered"] = df["Degrees/Certificates Offered"] + df["Degree"] + df["Degrees"] + df["Degree offered"] + df["Degree/Certification"] + df["Degrees/Certifications"]
df["Services"] = df["Services"] + df["Programs"] + df["Program"] + df["Instruments"] + df["Performance"] + df["Courses"]
df["Contact"] = df["Contact"] + df["Director"]
df["Additional locations"] = df["Additional locations"] + df["Additional Location"] + df["Additional location"] + df["Satellite Facility"] + df["Satellite locations"] + df["Studio locations"]
"""
df["Address/Phone"] = df["Address/Phone"] + df["Office"]
df["Specialty"] = df["Specialty"] + df["Specialties"]
df["Additional locations"] = df["Additional locations"] + df["Additional location"]
keep = ['Name', 'Address/Phone', 'Phone', 'Web', 'Email', 'Fax', 'Mail', 'Contact', 'Specialty',
        'Additional locations', 'Practice Areas']
df = df.loc[:, keep]
df = df.iloc[1:, :]
df.to_csv(output_filepath)
df

#extract_text(input_filepath)

,Name,Address/Phone,Phone,Web,Email,Fax,Mail,Contact,Specialty,Additional locations,Practice Areas
1,"ADAMS AND REESE, LLP","Regions Harbert Plaza 1901 6th Ave. N., Ste....",,msmcnairlaw.com,msm@mcnair.com,,,Michael S. McNair,Contracts,"RSA Battle House Tower 11 N. Water St., Ste....",
2,"DAVIS WRIGHT TREMAINE, LLP","188 W. Northern Lights Blvd. Anchorage, AK 9...",,dwt.com,jondawon@dwt.com,,,Jon Dawon,Litigation,,
3,DORSEY & WHITNEY LLP,"1031 W. 4th Ave., Ste. 600 Anchorage, AK 995...",,dorsey.com,mills.mike@dorsey.com,,,"Michael Mills, Partner",Intellectual Property,,
4,PERKINS COIE,"1029 W. 3rd Ave., Ste. 300 Anchorage, AK 995...",,perkinscoie.com,sGillstrm@perkinscoie.com,,,"Sarah C. Gillstrom, Partner",LItigation,,
5,"LEWIS ROCA ROTHGERBER CHRISTIE, LLP","201 East Washington St., Suite 1200 Phoenix,...",,lewisroca.com,mcurry@lewisroca.com,,,"Michael J.Curry, Partner","Intellectual Property, Trademarks and Copyrights",,
...,...,...,...,...,...,...,...,...,...,...,...
287,"BOWLES RICE, ATTORNEYS AT LAW","University Town Centre 125 Granville Square,...",,bowlesrice.com,,,,,Intellectual Property,"101 S. Queen St. Martinsburg, WV 25401 304-...",
288,FROST BROWN TODD,"500 Virginia St. East, Suite 1100 Charleston...",,frostbrowntodd.com,,,,,"Trademark Law, Intellectual Property",,
289,"BOARDMAN & CLARK, LLP","1 S. Pinckney St., Ste. 410 Madison, WI 5370...",,boardmanclark.com,,,,,"Patents, Copyrights, Trademarks and Trade Secr...","1038 Lincoln Ave. P.O. Box 87 Fennimore, WI...",
290,PERKINS COIE,"33 E. Main St., Ste. 201 Madison, WI 53703-5...",,perkinscoie.com,danstaett@perkinscoie.com,,,"David L. Anstaett, Partner","Trademark, Intellectual Property",,


---

## Aug23_Gear_Guide

In [359]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) == 5):
                                    if (span.get("flags", 0) != 20 and span['text'] != ' ') or (':' in span["text"]) or (not span["text"].isupper() and span['text'] != ' '):
                                        append_hash = False
                                    if (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        line_text += span["text"].strip() + ": " 
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip() + "\n"
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    #extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    for entry in extracted_list:
        entry = entry.strip('\n\n')
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                dict["Name"] = item.strip()
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list

In [365]:
"""
For Aug23_Gear_Guide.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "Aug23_Gear_Guide.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "Aug23_Gear_Guide.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

df["Web"] = df["Web"] + df["W eb"]
df["Additional locations"] = df["Additional locations"] + df["Additional location"]
keep = ['Name', 'Address/Phone', 'Email', 'Web', 'Contact', 'Studio Equip',
       'Musical Equip', 'Lighting', 'Stages', 'Additional locations',
       'Cartage', 'FX', 'Technical Services', 'Lighting Equip', 'FX Equip',
       'Cartage Equip', 'Special Services', 'Note', 'Basic Rate', 
       'Tech Services', 'Services', 'Musical Equipment',
       'Video', 'Tech services', 'Rooms', 'Equipment', 'Fax']
df = df.loc[:, keep]
df = df.iloc[1:, :]
df.to_csv(output_filepath)
df

#extract_text(input_filepath)

,Name,Address/Phone,Email,Web,Contact,Studio Equip,Musical Equip,Lighting,Stages,Additional locations,...,Note,Basic Rate,Tech Services,Services,Musical Equipment,Video,Tech services,Rooms,Equipment,Fax
1,ADVANCED AUDIO & VIDEO,"Box 2466 Huntsville, AL 35804 256-319-3030",tucker@advancedaudio.tv,advancedaudio.tv,,,,,,,...,,,,,,,,,,
2,HOLT AV,"401 28th St. S. Birmingham, AL 35233 800-32...",info@holtav.com,holtav.com,,,,,,,...,,,,,,,,,,
3,ACTION VIDEO PRODUCTIONS,"1231 W Northern Lights Blvd, Suite 2 Anchora...",,actvid1.com A RIZONA,,,,,,,...,,,,,,,,,,
4,FAV,Ford Audio Video 920 East Madison Street Ph...,,fordav.com,,,,,,,...,,,,,,,,,,
5,MEE,Music Equipment Rentals 5221 W Surrey Ave. ...,meerental@gmail.com,meerentals.com,Sam DeMarco,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,STUDIO INSTRUMENT RENTALS,"410 South 96th Street Unit 5 Seattle, WA 981...",,sir-usa.com wisconsin,,,,,,,...,,,,,,,,,,
211,"FULL COMPASS SYSTEMS, LTD.","9770 Silicon Prairie Pkwy. Madison, WI 53593...",rentals@fullcompass.com,fullcompass.com,,,,,,,...,,,,,,,,,,
212,INTELLASOUND PRODUCTIONS,"416 Venture Ct., Ste. 2 Verona, WI 53593-18...",mail@intellasound.com,intellasound.com,,,,,,,...,,,,,,,,,,
213,DGW VIDEO PRODUCTIONS LLC,"3537 Agate Rd. Cheyenne, WY 82009 307-529-1...",sales@dgwvideo.com,dgwvideo.com,,,,,,,...,,,,,,,,,,


---

## Sept23_FilmTV_Guide

In [369]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) == 6.25):
                                    if (span.get("flags", 0) != 20 and span['text'] != ' ') or (':' in span["text"]) or (not span["text"].isupper() and span['text'] != ' '):
                                        append_hash = False
                                    if (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        line_text += span["text"].strip() + ": " 
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip() + "\n"
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    #extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    for entry in extracted_list:
        entry = entry.strip('\n\n')
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                dict["Name"] = item.strip()
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list

In [376]:
"""
For Sept23_FilmTV_Guide.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "Sept23_FilmTV_Guide.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "Sept23_FilmTV_Guide.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

df["Additional locations"] = df["Additional locations"] + df["Additional location"] + df["Additional Location"]
df["Basic Rate"] = df["Basic Rate"] + df["Basic rate"]
df["Music Supervisors"] = df["Music Supervisors"] + df["Music Supervisor"]
df["Contact"] = df["Contact"] + df["Contacts"]
df["How to Submit"] = df["How to Submit"] + df["How to Contact"]
keep = ['Name', 'Address/Phone', 'Email', 'Contact', 'How to Submit', 'Web',
       'Music Supervisors', 'Credits', 'Comments', 'Cell', 'Styles',
       'Clients', 'Basic Rate', 'Additional locations']
df = df.loc[:, keep]
df = df.iloc[1:, :]
df.to_csv(output_filepath)
df

#extract_text(input_filepath)

,Name,Address/Phone,Email,Contact,How to Submit,Web,Music Supervisors,Credits,Comments,Cell,Styles,Clients,Basic Rate,Additional locations
1,1 SOUNDTRACK CENTRAL,5062 Lankershim Blvd. #174 N. Hollywood CA 9...,info@SoundtrackCentral.net,Jay Warsinske & Madeleine Smith,by emal only,,,,,,,,,
2,35 SOUND,"P.O. Box 217 Pacific Palisades, CA 90272 31...",gmarq@35sound.com,,no unsolicited material,35sound.com,"G. Marq Roswell, Carter LIttle","Man With the Iron Fists, Half the Sky, Birth S...",,,,,,
3,5 ALARM,"2140 N. Hollywood Way, #11028 Burbank, CA 91...",terrilynn@rescuerecords.net,,,5alarmmusic.com,,,,,,,,
4,AIR-EDEL USA,"1416 N. La Brea Ave. Hollywood, CA 90028 32...",,,does not accept unsolic- ited composer submiss...,airedel.com,,see web,,,,,,"Air-Edel Associates Ltd. 18 Rodmarton St., ..."
5,ARENA WAVES,"2764 Pleasant Rd, Suite A-625 Fort Mill, SC ...",info@arenawaves.com,,We welcome all styles of music. Please read su...,arenawaves.com,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,TOWN & COUNTRY CREATIVE,917-287-6575,rosanne@townandcountrycreative.com,Rosanna Herrick,,townandcountrycreative.com,,,,,,,call for info,
91,YESSIAN,"137 5th Ave., 3rd Fl. New York, NY 10010 21...",info-la@yessian.com Schulterblatt 58 20357 Ha...,,,yessian.com/services/musicsupervision,,,,,,,call for info,"33117 Hamilton Ct. Farmington Hills, MI 4833..."
92,FERRALLI STUDIOS,"2450 Dutch Rd Fairview, PA 16415 814-528-54...",info@ferralli-studios.com,,,ferralli-studios.com,,,,,,,call for info,
93,CONTINENTAL FILM PRODUCTIONS,"1466 Riverside Dr., Ste. E Chattanooga, TN 3...",,,,continentalfilm.com,,,,,,,call for info,


---

## Aug23_Rehearsal_Guide

In [386]:
def extract_text(pdf_path, y_min=52, y_max=762):
    """
    Extracts text from a PDF within a specific y-coordinate range.
    
    :param pdf_path: Path to the PDF file.
    :param y_min: Minimum y-coordinate boundary.
    :param y_max: Maximum y-coordinate boundary.
    :return: A string containing the extracted text.
    """
    text = ""
    doc = fitz.open(pdf_path)
    prev_line_appended = False
    
    for page in doc:
        text_dicts = page.get_text("dict")["blocks"]
        for text_dict in text_dicts:
            # Check if the block is a text block
            if text_dict.get("type") == 0:  # Type 0 indicates text
                _, y0, _, y1 = text_dict["bbox"]
                if y_min <= y0 <= y_max and y_min <= y1 <= y_max:  # Ensure y-coordinates are within the specified range
                    if "lines" in text_dict:  # Ensuring there are lines of text
                        for line in text_dict["lines"]:
                            span_count = 0
                            append_hash = True
                            line_text = ""
                            for span in line["spans"]:
                                if (span.get("color", 0) != 16777215) and (span.get("size", 0) == 5.75):
                                    if (span.get("flags", 0) != 20 and span['text'] != ' ') or (':' in span["text"]) or (not span["text"].isupper() and span['text'] != ' '):
                                        append_hash = False
                                    if (':' in span["text"]) and (span.get("flags", 0) == 20):
                                        line_text += span["text"].strip() + ": " 
                                    else:
                                        line_text += span["text"] + " " 
                                else:
                                    break  # No need to check further spans in this line
                                span_count += 1

                            if append_hash and not prev_line_appended and span_count == len(line["spans"]):
                                # Only prepend "##" if all conditions are met and it wasn't appended to the previous line
                                text += "##" + line_text.strip() + "\n"
                                prev_line_appended = True
                            else:
                                text += line_text.strip() + "\n"
                                prev_line_appended = False  # Reset for the next line
                            text += "\n"
    doc.close()
    extracted_text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove unicode
    #extracted_list = re.findall(r"##(.*?)##", extracted_text, re.DOTALL)
    extracted_list = extracted_text.split('##')
    return extracted_list


def get_dict(extracted_list):
    result_list = []
    for entry in extracted_list:
        entry = entry.strip('\n\n')
        entry_list = entry.split('\n')
        dict={}
        curr_attribute = ""
        for i, item in enumerate(entry_list):
            if i == 0:
                dict["Name"] = item.strip()
            elif '::' in item:
                item_split = item.split('::')
                curr_attribute = item_split[0].strip()
                curr_attribute = re.sub(r'\s+', ' ', curr_attribute)
                dict[curr_attribute] = item_split[1].strip()
            elif curr_attribute:
                dict[curr_attribute] += item.strip() + " "
            else: 
                curr_attribute = "Address/Phone"
                dict[curr_attribute] = item + " "
        result_list.append(dict)
    return result_list

In [387]:
"""
For Aug23_Rehearsal_Guide.pdf

# General structure:
    - Studio name in bold and all uppercase
    - Followed by address (optional)
    - Followed by phone number (optional)
    - Followed by other info in the format of "<attribute>:"
"""

script_dir = os.path.dirname(os.getcwd())
input_filepath = os.path.join(script_dir, "Aug23_Rehearsal_Guide.pdf")
output_filepath = os.path.join(script_dir, "scraping", "scraped_data", "company_data", "Aug23_Rehearsal_Guide.csv")

df = pd.DataFrame.from_dict(get_dict(extract_text(input_filepath)))
df.fillna('', inplace=True)

df["Basic Rate"] = df["Basic Rate"] + df["Basic rate"] + df["Rates"]
df['Additional locations'] = df['Additional locations'] + df['Additional location'] + df['Additional Locations'] + df['Other Locations']
df['Services'] = df['Services'] + df['Special Services']
df['Address/Phone'] = df['Address/Phone'] + df["Address"]
df["Web"] = df["Web"] + df["Website"]
keep = ['Name', 'Address/Phone', 'Phone', 'Email', 'Web', 'Basic Rate',
       'Rooms', 'Contact', 'Services',
       'Additional locations', 'Equipment', 'Clients', 
       'Studios']
df = df.loc[:, keep]
df = df.iloc[1:, :]
df.to_csv(output_filepath)
df

#extract_text(input_filepath)

,Name,Address/Phone,Phone,Email,Web,Basic Rate,Rooms,Contact,Services,Additional locations,Equipment,Clients,Studios
1,"GUEST HOUSE STUDIOS, LLC","641 Mockingbird Ln. Eclectic, AL 36024 334-...",,ghs@guesthousestudios.com,guesthousestudios.com,Email or call for rates,,,,"5029 Edmondson Pike Nashville, TN 629-267-...",,,
2,"ALMEC, LLC","1460 Ann St. Montgomery, AL 36107 334-649-2...",,todd@souvignier.com,facebook.com/almecllc,please call/email for info,"wide range, from single-user to multi-room sui...",,,,,,
3,SCOTTSDALE STUDIOS,"Scottsdale, AZ 85258 480-767-3777 Email : ...",,rockstar@sersoundworks.com,sersoundworks.com,please call for info,,,,,,,
4,S.I.R. PHOENIX,"3541 East Broadway Road Phoenix, AZ 85040 4...",,,sir-usa.com,please call for info,,,,,,,
5,ANNEX REHEARSAL STUDIOS,"2554 Grant Ave. San Lorenzo, CA 94580 510-9...",,info@annexrehearsalstudios.com,facebook.com/AnnexRSLive,please call for info,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,SEATTLE DRUM SCHOOL,"12729 Lake City Way NE Seattle, WA 98125 20...",,seattledrumgeorgetown@gmail.com 206-763-9700,seattledrumschool.com,$35 per half hour and $65 per one hour lesson,,,,"1010 S. Bailey St. Seattle, WA 98108",,,
146,SEATTLE REHEARSAL,"2424 1st Ave. S. Seattle, WA 98134 206-287-...",,jodiopitz@yahoo.com,seattlerehearsal.com,please call for info,,Jodi,,,,,
147,S.I.R. SEATTLE,"410 South 96th Street Unit 5 Seattle, WA 98...",,wainfo@sir-usa.com,sir-usa.com,please call for info,,,,,,,
148,MADISON MUSIC FOUNDRY,"2818 Index Rd. Fitchburg, WI 53713 608-270-...",,info@madisonmusicfoundry.com,madisonmusicfoundry.com,call for rates,,,,,,,


In [388]:
df.columns

Index(['Name', 'Address/Phone', 'Phone', 'Email', 'Web', 'Basic Rate', 'Rooms',
       'Contact', 'Services', 'Additional locations', 'Equipment', 'Clients',
       'Studios'],
      dtype='object')

## Text analysis

In [381]:
# INITIAL TEXT ANALYSIS

text = ""
doc = fitz.open(input_filepath)

for page in doc:
    for text_block in page.get_text("dict")["blocks"]:
        if text_block['type'] == 0:  # Text block
            for line in text_block["lines"]:
                for span in line["spans"]:  # Iterate over spans
                    print(f"Text: {span['text']}, Color: {span['color']}, Size: {span['size']}, Flags: {span['flags']}")
            print()
                    #print(span)
                    # Examine span properties for clues about visibility


Text: C, Color: 16777215, Size: 8.0, Flags: 4
Text: O M, Color: 16777215, Size: 8.0, Flags: 4
Text: P I, Color: 16777215, Size: 8.0, Flags: 4
Text: L E D  B Y  D E N, Color: 16777215, Size: 8.0, Flags: 4
Text: I S, Color: 16777215, Size: 8.0, Flags: 4
Text: E  C, Color: 16777215, Size: 8.0, Flags: 4
Text: O S, Color: 16777215, Size: 8.0, Flags: 4
Text: O, Color: 16777215, Size: 8.0, Flags: 4

Text: 50 August 2023   , Color: 2236191, Size: 9.0, Flags: 20
Text:    , Color: 2236191, Size: 9.0, Flags: 20
Text: musicconnection.com, Color: 2236191, Size: 9.0, Flags: 4

Text: Now expanded for 2023, this is an exclusive list of professional rehearsal studios. All information below is verified by the listees., Color: 2236191, Size: 8.772114753723145, Flags: 6

Text: To receive thousands of free industry contacts, visit musicconnection.com/industry-contacts., Color: 2236191, Size: 8.772114753723145, Flags: 6

Text: ALABAMA, Color: 16777215, Size: 7.155417442321777, Flags: 4

Text: GUEST HOUSE ST